# Export of Baseline PSTH Data

* Separate exports for acute and one for chronic experiments.
* Datasets contain spiking for each neuron aligned to shock times

Datasets exported:

- **acute_baseshock_raw.csv**
- **acute_baseshock_psth.csv**
- **chronic_baseshock_raw.csv**
- **chronic_baseshock_psth.csv**

In [13]:
from ephys_queries import select_spike_times, select_discrete_data, select_neurons
from ephys_queries import db_setup_core
import dotenv

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path 

In [11]:
from spiketimes.df import align_df, align_df_by

In [4]:
dotenv.load_dotenv()
engine, metadata = db_setup_core()

In [5]:
data_dir = Path(".").absolute().parent / "data"
df_clusters = pd.read_csv(data_dir / "baseline.csv")

## Acute Data

#### Select Raw Data

In [27]:
group_names = [
    "acute_citalopram", 
    "acute_saline", 
    "acute_cit", 
    "acute_sal"
]

block_name = "base_shock"
t_before = 10


neurons = select_neurons(
            engine,
            metadata,
            group_names=group_names
).rename(columns={"id":"neuron_id"})


df_spikes_acute = (
    select_spike_times(
            engine, 
            metadata,
            group_names=group_names,
            block_name=block_name,
            )
    .pipe(lambda x: pd.merge(x, neurons[["neuron_id", "session_name"]]))
    .pipe(lambda x: x.assign(spiketimes=x["spike_time_samples"].divide(30000)))
)

df_events_acute = (
    select_discrete_data(
            engine,
            metadata,
            group_names=group_names,
            block_name=block_name
    )
    .pipe(lambda x: x.assign(event_s=x["timepoint_sample"].divide(30000)))
)

#### Align Data to Events

In [69]:
aligned_acute = (
    align_df_by(
        df_data=df_spikes_acute, 
        df_data_data_colname="spiketimes", 
        df_data_group_colname="session_name",
        df_events=df_events_acute,
        df_events_event_colname="event_s",
        df_events_group_colname="session_name", 
        t_before=0.2,
        max_latency=1.8,
    )
    .dropna()
    .pipe(lambda x: pd.merge(df_clusters[["neuron_id", "cluster"]], x))
)

/home/rory/repos/spiketimes/spiketimes/df/alignment.py:74: UserWarning: Groups in df_events and df_data are not identical
  warnings.warn("Groups in df_events and df_data are not identical")
/home/rory/repos/spiketimes/spiketimes/alignment/alignment.py:149: RuntimeWarning: invalid value encountered in greater_equal
  (negative_latencies >= (t_before * -1)),
/home/rory/repos/spiketimes/spiketimes/alignment/alignment.py:157: RuntimeWarning: invalid value encountered in greater
  latencies[latencies > max_latency] = np.nan


#### Construct PSTH

In [68]:
bins = np.round(np.arange(-0.2, 1.80, 0.02), 4)

psth_acute = (
    bin_df(aligned_acute, colname="aligned", bins=bins, bin_val_name="bin")
    .groupby(["neuron_id", "bin"])
    .apply(lambda x: len(x))
    .unstack()
    .reset_index()
    .melt(id_vars="neuron_id", value_name="counts")
    .sort_values(by=["neuron_id", "bin"])
    .reset_index(drop=True)
    .fillna(0)
    .pipe(lambda x: pd.merge(x, df_clusters[["neuron_id", "session_name", "cluster", "group_name"]]))
)

#### Save Data

In [70]:
aligned_acute.to_csv(data_dir / "acute_baseshock_raw.csv", index=False)
psth_acute.to_csv(data_dir / "acute_baseshock_psth.csv", index=False)

## Chronic Data

#### Select Raw Data

In [71]:
group_names = ["citalopram_continuation", "chronic_saline", "citalopram_discontinuation", 
               "chronic_citalopram", "chronic_saline_"]

block_name = "base_shock"
t_before = 10


neurons = select_neurons(
            engine,
            metadata,
            group_names=group_names
).rename(columns={"id":"neuron_id"})


df_spikes_chronic = (
    select_spike_times(
            engine, 
            metadata,
            group_names=group_names,
            block_name=block_name,
            )
    .pipe(lambda x: pd.merge(x, neurons[["neuron_id", "session_name"]]))
    .pipe(lambda x: x.assign(spiketimes=x["spike_time_samples"].divide(30000)))
)

df_events_chronic = (
    select_discrete_data(
            engine,
            metadata,
            group_names=group_names,
            block_name=block_name
    )
    .pipe(lambda x: x.assign(event_s=x["timepoint_sample"].divide(30000)))
)

#### Align Data to Events

In [72]:
aligned_chronic = (
    align_df_by(
        df_data=df_spikes_chronic, 
        df_data_data_colname="spiketimes", 
        df_data_group_colname="session_name",
        df_events=df_events_chronic,
        df_events_event_colname="event_s",
        df_events_group_colname="session_name", 
        t_before=0.2,
        max_latency=1.8,
    )
    .dropna()
    .pipe(lambda x: pd.merge(df_clusters[["neuron_id", "cluster"]], x))
)

/home/rory/repos/spiketimes/spiketimes/df/alignment.py:74: UserWarning: Groups in df_events and df_data are not identical
  warnings.warn("Groups in df_events and df_data are not identical")
/home/rory/repos/spiketimes/spiketimes/alignment/alignment.py:149: RuntimeWarning: invalid value encountered in greater_equal
  (negative_latencies >= (t_before * -1)),
/home/rory/repos/spiketimes/spiketimes/alignment/alignment.py:157: RuntimeWarning: invalid value encountered in greater
  latencies[latencies > max_latency] = np.nan


#### Construct PSTH

In [73]:
bins = np.round(np.arange(-0.2, 1.80, 0.02), 4)

psth_chronic = (
    bin_df(aligned_chronic, colname="aligned", bins=bins, bin_val_name="bin")
    .groupby(["neuron_id", "bin"])
    .apply(lambda x: len(x))
    .unstack()
    .reset_index()
    .melt(id_vars="neuron_id", value_name="counts")
    .sort_values(by=["neuron_id", "bin"])
    .reset_index(drop=True)
    .fillna(0)
    .pipe(lambda x: pd.merge(x, df_clusters[["neuron_id", "session_name", "cluster", "group_name"]]))
)

#### Save Data

In [74]:
aligned_chronic.to_csv(data_dir / "chronic_baseshock_raw.csv", index=False)
psth_chronic.to_csv(data_dir / "chronic_baseshock_psth.csv", index=False)